In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from db_utils import RDSDatabaseConnector 
from transform_data import DataTransform
from get_information import DataFrameInfo
import missingno as msno
from plotter import Plotter
import seaborn as sns
import plotly.express as px
from statsmodels.graphics.gofplots import qqplot
from scipy import stats
pd.options.mode.chained_assignment = None

In [ ]:
credentials = RDSDatabaseConnector.yaml_creds_loader()
database = RDSDatabaseConnector(credentials)
database.save_df_to_csv()

In [ ]:
loan = pd.read_csv('loan_payments.csv')
pd.set_option('display.max_columns', None)
loan

# Converting Column Datatypes
a lot of these columns are not formatted to the correct dtype which you can see from the loan.info() below. To remedy this I will convert a lot of these columns which can be formatted correctly using my transform_data module

In [ ]:
loan.info()

In [ ]:
DataTransform.convert_columns(loan)

In [ ]:
convert_term_dict = {'36 months' : 36, '60 months' : 60}
loan['term'] = loan['term'].replace(convert_term_dict) # replacing string to flaot

In [ ]:
convert_emp_length_dict = {
    '< 1 year' : 0,
    '1 year' : 1,
    '2 years' : 2,
    '3 years' : 3,
    '4 years' : 4,
    '5 years' : 5,
    '6 years' : 6,
    '7 years' : 7,
    '8 years' : 8,
    '9 years' : 9,
    '10 years' : 10,
    '10+ years' : 11
}
loan['employment_length'] = loan['employment_length'].replace(convert_emp_length_dict)

# Dataframe information
in this segment I will be looking into the dataframe itself and try and generate useful information about the DataFrame

In [ ]:
DataFrameInfo.datatypes(loan)

In [ ]:
DataFrameInfo.statistics(loan)

In [ ]:
DataFrameInfo.percent_null(loan)

In [ ]:
Plotter.null_matrix(loan)

# Dropping Columns
---
### Null values >= 50%
All the columns with >= 50% null values will be dropped as they will introduce too much noise which will not help with analysis.
These Columns include: **mths_since_last_delinq,** **mths_since_last_record**, **next_payment_date** and **mths_since_last_major_derog**

---

We can see **Unnamed: 0** column is equal to the id, so we can safely drop this column and just refer to the id.

---
### Static Value.

The policy_code column is just filled with 1, as this column will not contribute to any findings we can safely drop this column.

### non important

the id and member_id columns are random numbers assigned to the loan/customer, thus they will not be important in our analysis to find insights, these will be dropped




In [ ]:
DataTransform.drop_columns(loan, ['mths_since_last_delinq', 'mths_since_last_record', 'next_payment_date', 'mths_since_last_major_derog'])
DataTransform.drop_columns(loan, ['Unnamed: 0', 'policy_code'])
DataTransform.drop_columns(loan, ['id','member_id'])
loan.shape

In [ ]:
Plotter.null_matrix(loan)

In [ ]:
null_values = DataFrameInfo.get_null_greater_than_zero(loan)

print(null_values, "\n\n")
print(loan[null_values.index].nunique()) # unique values for columns with null values

we can see the "term", "employment_length", and "collections_12_mths_ex_med" are clearly categorical so lets look at their values.

In [ ]:
print(
    loan.term.unique(),
    loan.employment_length.unique(),
    loan.collections_12_mths_ex_med.unique()
)

lets take a closer look at the 'term' column

In [ ]:
DataFrameInfo.count_values(loan, 'term')


there is more 36 month terms than 60 month terms, however they seem to be relatively close, so as not to introduce any bias, i shall drop any rows with these null values.


In [ ]:
DataTransform.drop_null_rows_from_columns(loan, ['term'])

In [ ]:
DataFrameInfo.get_null_greater_than_zero(loan)

employment_length contains values for the customer employment length.
We should remove any null values for this column as not to introduce any bias.

In [ ]:
DataTransform.drop_null_rows_from_columns(loan,['employment_length'])

last_payment_date, last_credit_pull_date and collections_12_mths_ex_med have very little null values so lets just drop those rows

In [ ]:
DataTransform.drop_null_rows_from_columns(loan,['last_payment_date', 'last_credit_pull_date', 'collections_12_mths_ex_med'])


In [ ]:
DataFrameInfo.get_null_greater_than_zero(loan)

# Remaining Null Values

lets take a closer look at the int_rate to possible impute these values.

In [ ]:
# we know from earlier that inr_rate is not categorical.
Plotter.plot_boxplot(loan['int_rate'])

In [ ]:
Plotter.plot_boxplot(loan['funded_amount'])

### Outliers
There are a lot of outliers in the interest rates, we will not get rid of them yet. To remove the null values it would therefore be sensible to impute the median values so the outliers have a lesser effect.

In [ ]:
loan['int_rate'].fillna(loan['int_rate'].median(), inplace=True)
loan['funded_amount'].fillna(loan['funded_amount'].median(), inplace=True)

DataFrameInfo.get_null_greater_than_zero(loan)

In [ ]:
Plotter.null_matrix(loan)

We no longer have any null values in our DataFrame

# Skewness

in this section we will check for skewness and attempt to transform the skewed data inro a normal distribution.

In [ ]:
loan.skew(numeric_only=True)

### what columns will we transform

since total_rec_late_fee, recoveries, delinq_2yrs, out_prncp, our_prncp_inv, collection_recovery_fee, collections_12_mths_ex_med have some inherent bias, we will not transform these columns.

term has only 2 values so we dont ahve to worry about that.


otherwise any |skewness| > 0.5 will be transformed

In [ ]:
transform = [
    'loan_amount',
    'funded_amount',
    'funded_amount_inv',
    'instalment',
    'annual_inc',
    'inq_last_6mths',
    'open_accounts',
    'total_accounts',
    'total_payment',
    'total_payment_inv',
    'total_rec_prncp',
    'total_rec_int',
    'last_payment_amount',
]
loan['loan_amount'].hist(bins=60)


In [ ]:
qqplot_annual_inc = qqplot(loan['loan_amount'], line='q', fit=True)

we can see a positive skew due to over representation of smaller values and under representation of larger values.

lets use a log transform to try normalise the annual income column.

In [ ]:
loan_clean = loan.copy() # creating a copy of our dataframe so we can go back to older data
loan_amount_transformed = DataTransform.log_transform(loan_clean, 'loan_amount')
loan_amount_transformed.hist(bins=40)

In [ ]:
loan_amount_transformed.skew()

we want to keep |skewness| < 0.5 so lets try a boxcox transformation.

In [ ]:
loan_amount_transformed = DataTransform.boxcox_transform(loan, 'loan_amount', lmbda=0.4) # try a lambda value of 0.4

In [ ]:
loan_amount_transformed.hist(bins=60)
loan_amount_transformed.skew()

the boxcox gets us below our 0.5 threshold so we shall keep it like that

In [ ]:
loan_clean['loan_amount'] = loan_amount_transformed

lets log transform the rest of the columns to see if it gets below our threshold, otherwise we can try a boxcox transform.

In [ ]:
transform.remove('loan_amount') # dropping 'loan_amount'

for column in transform:
    transformed = DataTransform.log_transform(loan, column)
    print(f"{column}: {transformed.skew()}")

annual_inc, open_accounts and last_payment_amount are within are threshold and can stay with the log transformation.

In [ ]:
loan_clean['annual_inc'] = DataTransform.log_transform(loan, 'annual_inc')
loan_clean['open_accounts'] = DataTransform.log_transform(loan, 'open_accounts')
loan_clean['last_payment_amount'] = DataTransform.log_transform(loan, 'last_payment_amount')

loan_clean[transform].skew()

now we can attempt to do a boxcox transform on the columns, first we'll use a lambda value of 0.3 and see if that gets any columns below our threshold

In [ ]:
transform.remove('annual_inc')
transform.remove('open_accounts')
transform.remove('last_payment_amount')

In [ ]:
for column in transform:
    transformed = DataTransform.boxcox_transform(loan, column, 0.3)
    print(f"{column}: {transformed.skew()}")

we can see that brings a lot of columns into our threshold for skewness, so lets update those columns with lambda= 0.3 and look further into the rest

In [ ]:
for column in transform:
    loan_clean[column] = DataTransform.boxcox_transform(loan, column, 0.3)

In [ ]:
loan_clean.skew(numeric_only=True)

In [ ]:
loan_clean.to_csv('loans_transformed.csv')

this is roughly in line with what we want to see.

# Removing Outliers

in this section we will be removing any outliers that will affect the data, to do this I will first plot some boxplots for each numerical column to see the different outliers

In [ ]:
boxplot_columns = [
'loan_amount',
'funded_amount',
'funded_amount_inv',
'int_rate',
'instalment',
'employment_length',
'annual_inc',
'dti',
'inq_last_6mths',
'open_accounts',
'total_accounts',
'total_payment',
'total_payment_inv',
'total_rec_prncp',
'total_rec_int',
'last_payment_amount',
] # all columns that dont have 0 as upper and lower fence


## identifying outliers

In [ ]:
Plotter.plot_boxplot(loan_clean[boxplot_columns])

there are a lot of outliers here though it seems loan_amount is pretty stable lets replot without loan_amount to see a better picture


In [ ]:
boxplot_columns.remove('loan_amount')
Plotter.plot_boxplot(loan_clean[boxplot_columns])

we can see a lot of outliers here, lets take a closer look at annual_inc and open_accounts, and then identify the outlier values

In [ ]:
Plotter.plot_boxplot(loan_clean[['annual_inc', 'open_accounts']])

In [ ]:
for column in boxplot_columns:
    print(DataFrameInfo.find_outliers(loan_clean, column))
    print("\n\n")

lets proceed to remove these outliers

In [ ]:
for column in boxplot_columns:
    DataTransform.remove_outliers(loan_clean, column)

now the outliers are removed lets replot the box plots to see how it looks.

In [ ]:
Plotter.plot_boxplot(loan_clean[boxplot_columns])

In [ ]:
Plotter.plot_boxplot(loan_clean[['annual_inc','open_accounts']])

the outliers are removed

# Colinearity

in this section we will calculate the correlation matrix and proceed to find any colinear variables and drop any which is too high, 

In [ ]:
correlation_matrix = loan.corr(numeric_only=True)

Plotter.correlation_matrix(correlation_matrix)

What can we see from the matrix:
- out_prncp and out_prncp_inv are VERY similar with a correlation of 1 (we are going to drop one of these)
- there is multicollinearity between loan_amount, funded_amount and funded_amount_inv, it will be safe to drop one or 2 of these variables
- there is multicollinearity between total_payment, total_payment_inv and total_rec_prncp, it will be safe to drop one or 2 of these variables
- there is strong co linearity between recoveries and collection_recovery_fee, we can drop one of these variables.

based off the analysis of the matrix I will be dropping the following columns:
- out_prncp_inv
- funded_amount and funded_amount_inv
- total_payment_inv and total_rec_prncp
- collection_recovery_fee

In [ ]:
loan.drop(columns=['out_prncp_inv','funded_amount', 'funded_amount_inv', 'total_payment_inv', 'total_rec_prncp', 'collection_recovery_fee'], inplace=True)
loan_clean.drop(columns=['out_prncp_inv','funded_amount', 'funded_amount_inv', 'total_payment_inv', 'total_rec_prncp', 'collection_recovery_fee'], inplace=True)


In [ ]:
correlation_matrix = loan.corr(numeric_only=True)

Plotter.correlation_matrix(correlation_matrix)

a lot of the colinearity has been removed

# Analysis

### Current state of loans:


In [ ]:
loan_clean

In [ ]:
loan_clean['percentage_recovered'] = loan['total_payment']/loan['loan_amount']
loan_clean['recovered'] = loan_clean['percentage_recovered'].map(lambda x: True if x >= 1 else False)

In [ ]:
Plotter.plot_hist(loan_clean, 'percentage_recovered')

In [ ]:

loan_clean['recovered'].sum()/len(loan_clean)

55% of loans are paid off, leaving 45% unpaid, now lets take a look 6 months in the future and see how many of those below 100%(of the loan paid off) will be able to pay it off.

In [ ]:
loan_unpaid = loan[loan['total_payment'] < loan['loan_amount']] # select only unpaid loan records
loan_unpaid['total_payment'] = loan_unpaid['total_payment'] + 6*loan_unpaid['instalment'] # 6 months of payments
loan_unpaid['percentage_recovered'] = loan_unpaid['total_payment'] / loan_unpaid['loan_amount']
Plotter.plot_hist(loan_unpaid, 'percentage_recovered')

In [ ]:
loan_unpaid['recovered'] = loan_unpaid['percentage_recovered'].map(lambda x: True if x >= 1 else False)

In [ ]:
round(loan_unpaid['recovered'].sum()/len(loan_unpaid) * 100, 2)

36.6% will repay their loans within 6 months

## Calculated Loss
in this section we will calculate loss of loans

In [ ]:
loan_clean.groupby(['loan_status'], observed=False).loan_amount.count() # counts loan_status' in loan_amount

For simplicity those assigned as "Does not meet the credit policy will be assigned to Charged off or Fully paid respectively

In [ ]:
convert_loan_status_dict = {
    'Does not meet the credit policy. Status:Charged Off' : 'Charged Off',
    'Does not meet the credit policy. Status:Fully Paid' : 'Fully Paid'
}

loan['loan_status'] = loan['loan_status'].replace(convert_loan_status_dict) # main df

loan_clean['loan_status'] = loan_clean['loan_status'].replace(convert_loan_status_dict) # cleaned df

In [ ]:
loan_clean.groupby(['loan_status'], observed=False).loan_amount.count()


In [ ]:
loan_clean.groupby(['loan_status'], observed=False).loan_amount.count().plot.bar()

In [ ]:
round(loan_clean[loan_clean['loan_status'] == 'Charged Off']['loan_status'].count() / len(loan_clean) * 100, 2)

approximately 10% of loans are charged off

In [ ]:
loan_clean['total_revenue'] = loan['loan_amount'] * (1+loan['int_rate']/100)**(loan['term']/12)

In [ ]:
loan_clean['total_revenue'].sum()

In [ ]:
loan_clean[loan_clean['loan_status'] == 'Charged Off']['total_revenue'].sum()

the company has lost 110 154 776.78  in revenue from charged off loans